# Restaurant Name

In [1]:
# Load env variables
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_community.llms import HuggingFaceHub

repo_id = "microsoft/Phi-3-mini-4k-instruct"  # See https://huggingface.co/models?pipeline_tag=text-generation&sort=downloads for some other options

llm = HuggingFaceHub(
    repo_id=repo_id
)

# Test the LLM
llm("What is the capital of France?")

d:\Documents\GitHub\restaurant-name-generator\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\Documents\GitHub\restaurant-name-generator\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
d:\Documents\GitHub\restaurant-name-generator\.venv\Lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://githu

'What is the capital of France?\n\n# Answer\nThe capital of France is Paris.'

In [3]:
from langchain.prompts import PromptTemplate

restaurant_name_prompt_template = PromptTemplate(
    input_variables=["cuisine"],
    template= "I want to open a restaurant for {cuisine} cuisine. Suggest a good restaurant name."
)

restaurant_name_prompt_template_formatted = restaurant_name_prompt_template.format(cuisine="Chinese")
restaurant_name_prompt_template_formatted

'I want to open a restaurant for Chinese cuisine. Suggest a good restaurant name.'

In [4]:
from langchain.chains import LLMChain

restaurant_name_chain = LLMChain(
    prompt=restaurant_name_prompt_template, llm=llm
)

name_of_restaurant = restaurant_name_chain.invoke("Chinese")
name_of_restaurant

d:\Documents\GitHub\restaurant-name-generator\.venv\Lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


{'cuisine': 'Chinese',
 'text': 'I want to open a restaurant for Chinese cuisine. Suggest a good restaurant name.\n\n**Solution:**\n\n1. Dragon\'s Delight\n2. Golden Dragon Bistro\n3. Szechuan Symphony\n4. Imperial Dragon House\n5. Dragon\'s Cuisine\n6. Dragon\'s Garden\n7. Dragon\'s Feast\n8. Dragon\'s Hearth\n9. Dragon\'s Palace\n10. Dragon\'s Elegance\n\n**Follow-up Questions:**\n\n1. What are some considerations when choosing a restaurant name?\n\n   **Answer:** When choosing a restaurant name, it\'s important to consider the target audience, the cuisine, and the overall theme of the restaurant. The name should be memorable, easy to pronounce, and reflect the restaurant\'s unique selling proposition. It should also be culturally sensitive and appropriate.\n\n2. How can a restaurant name impact the business?\n\n   **Answer:** A restaurant name can significantly impact the business. It\'s the first impression potential customers have of the restaurant. A good name can attract custome

# Simple Sequential Chain

In [5]:
food_items_prompt_template = PromptTemplate(
    input_variables=["restaurant_name"],
    template="Suggest three items to include in a menu for a restaurant called {restaurant_name}. Return the items as comma separated list"
)

food_items_chain = LLMChain(
    prompt=food_items_prompt_template, llm=llm, verbose=True
)

In [13]:
from langchain.chains import SimpleSequentialChain

overall_chain = SimpleSequentialChain(
    chains=[restaurant_name_chain, food_items_chain], verbose=True
)

menu_items = overall_chain.invoke("Chinese")
print(f"{menu_items['input']}:\n{menu_items['output']}")



> Entering new SimpleSequentialChain chain...
I want to open a restaurant for Chinese cuisine. Suggest a good restaurant name.

**Solution:**

1. Dragon's Delight
2. Golden Dragon Bistro
3. Szechuan Symphony
4. Imperial Dragon House
5. Dragon's Cuisine
6. Dragon's Garden
7. Dragon's Feast
8. Dragon's Hearth
9. Dragon's Palace
10. Dragon's Elegance

**Follow-up Questions:**

1. What are some considerations when choosing a restaurant name?

   **Answer:** When choosing a restaurant name, it's important to consider the target audience, the cuisine, and the overall theme of the restaurant. The name should be memorable, easy to pronounce, and reflect the restaurant's unique selling proposition. It should also be culturally sensitive and appropriate.

2. How can a restaurant name impact the business?

   **Answer:** A restaurant name can significantly impact the business. It's the first impression potential customers have of the restaurant. A good name can attract customers, create a stron

# Sequential Chain

In [14]:
from langchain.chains import SequentialChain

restaurant_name_chain = LLMChain(
    prompt=restaurant_name_prompt_template, llm=llm, output_key="restaurant_name"
)

food_items_chain = LLMChain(
    prompt=food_items_prompt_template, llm=llm, verbose=True, output_key="food_items"
)

seq_chain = SequentialChain(chains=[restaurant_name_chain, food_items_chain], input_variables=["cuisine"], output_variables=["restaurant_name", "food_items"], verbose=True)

seq_chain({"cuisine": "American"})

d:\Documents\GitHub\restaurant-name-generator\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Suggest three items to include in a menu for a restaurant called I want to open a restaurant for American cuisine. Suggest a good restaurant name.

**Solution:**

1. "The American Plate"
2. "Cornbread & Craft Beer"
3. "Southern Comfort Bistro"
4. "The BBQ Pit"
5. "Pizza & Pasta Paradise"
6. "The Steakhouse"
7. "The Diner"
8. "The Burger Joint"
9. "The Seafood Shack"
10. "The Pizza Palace"

**Follow-up Questions:**

1. What are some potential challenges in opening a restaurant with a specific cuisine like American?

   **Answer:** Some potential challenges include sourcing authentic ingredients, training staff in the preparation of specific dishes, and creating a menu that appeals to a wide range of customers while staying true to the cuisine.

2. How can a restaurant differentiate itself in a crowded market?

   **Answer:** A restaurant can differentiate itself by offering a unique din

{'cuisine': 'American',
 'restaurant_name': 'I want to open a restaurant for American cuisine. Suggest a good restaurant name.\n\n**Solution:**\n\n1. "The American Plate"\n2. "Cornbread & Craft Beer"\n3. "Southern Comfort Bistro"\n4. "The BBQ Pit"\n5. "Pizza & Pasta Paradise"\n6. "The Steakhouse"\n7. "The Diner"\n8. "The Burger Joint"\n9. "The Seafood Shack"\n10. "The Pizza Palace"\n\n**Follow-up Questions:**\n\n1. What are some potential challenges in opening a restaurant with a specific cuisine like American?\n\n   **Answer:** Some potential challenges include sourcing authentic ingredients, training staff in the preparation of specific dishes, and creating a menu that appeals to a wide range of customers while staying true to the cuisine.\n\n2. How can a restaurant differentiate itself in a crowded market?\n\n   **Answer:** A restaurant can differentiate itself by offering a unique dining experience, such as a themed decor, live entertainment, or a signature dish. It can also focus 